## Image classification with Convolutional Neural Networks

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

from pathlib import Path

In [3]:
if os.name == 'nt':
    PATH = str(Path.home()) + "\\data\\dogscats\\"
else:
    PATH = "data/dogscats/"
    
sz=224
arch=vgg16
bs=64

In [4]:
# Uncomment the below if you need to reset your precomputed activations
if os.name == 'nt':
    ## warning: will remove everything under the following path (windows):
    #delPath = f"{PATH}tmp\\"
    #!rmdir /S /Q "{delPath}"
    ;
else:
    #!rm -rf {PATH}tmp
    ;

In [5]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))

In [6]:
arch = NASNetALarge

NameError: name 'NASNetALarge' is not defined

In [6]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [7]:
%time learn.fit(0.01, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.062126   0.03306    0.986946  
    1      0.047412   0.031187   0.987935                                                                              
    2      0.045277   0.029062   0.989122                                                                              

Wall time: 9.4 s


[0.02906198, 0.989121835443038]

In [8]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [9]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [10]:
learn.fit(1e-2, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.047355   0.031113   0.988133  
    1      0.052165   0.028257   0.988924                                                                              



[0.028257253, 0.9889240506329114]

In [11]:
learn.precompute=False

In [12]:
%time learn.fit(1e-2, 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.057572   0.028483   0.988924  

Wall time: 2min 23s


[0.028483355, 0.9889240506329114]

In [13]:
learn.unfreeze()

In [14]:
lr=np.array([1e-4,1e-3,1e-2])

In [15]:
%time learn.fit(lr, 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.059411   0.031233   0.987737  

Wall time: 4min 56s


[0.03123302, 0.9877373417721519]

In [16]:
%time learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.044555   0.028049   0.98932   
    1      0.03885    0.02506    0.990309                                                                              
    2      0.039219   0.024144   0.991297                                                                              
    3      0.042383   0.024482   0.992484                                                                              
    4      0.033613   0.026511   0.989715                                                                              
    5      0.02608    0.021637   0.993275                                                                              
    6      0.026381   0.021231   0.993275                                                                              

Wall time: 34min 46s


[0.021231495, 0.9932753164556962]

In [17]:
%time learn.fit(lr, 3, cycle_len=3)

HBox(children=(IntProgress(value=0, description='Epoch', max=9), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.036294   0.024299   0.991891  
    1      0.032373   0.021581   0.992867                                                                              
    2      0.020633   0.021741   0.992867                                                                              
    3      0.031901   0.023521   0.991285                                                                              
    4      0.019729   0.023443   0.992274                                                                              
 41%|███████████████████████████▍                                       | 128/313 [01:52<02:42,  1.14it/s, loss=0.0198]

KeyboardInterrupt: 

In [ ]:
%time log_preds_augmented,y = learn.TTA()
log_preds = log_preds_augmented.mean(axis=0)
accuracy(torch.from_numpy(log_preds), torch.from_numpy(y))

In [ ]:
NASNetALarge 